In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'C:/Users/bhats/Desktop/NSG ML/genres/{g}'):
        songname = f'C:/Users/bhats/Desktop/NSG ML/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [4]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'C:/Users/bhats/Desktop/NSG ML/genres/{g}'):
        songname = f'C:/Users/bhats/Desktop/NSG ML/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


In [ ]:
data = pd.read_csv('data.csv')
data.head()

In [7]:
data.shape

(1000, 28)

In [8]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [9]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [76]:
len(y_train)

800

In [77]:
len(y_test)

200

In [78]:
X_train[10]

array([ 0.44421636,  1.10465319,  0.88901755,  1.52678374,  1.21601861,
        0.44346947,  0.72196367, -0.58255056,  1.18285388, -1.65753694,
        1.25332668, -1.32482862,  1.80897954, -1.01804305,  1.22454192,
       -0.57680451,  0.37345806, -0.83616372,  0.66618714, -1.15500739,
        0.5706717 ,  0.13800492,  1.64068314,  1.39655122,  2.31061819,
        0.92296964])

In [79]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [80]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [81]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
800/800 [==============================] - 1s 1ms/step - loss: 2.1671 - acc: 0.2500
Epoch 2/20
800/800 [==============================] - 0s 39us/step - loss: 1.8335 - acc: 0.3438
Epoch 3/20
800/800 [==============================] - 0s 40us/step - loss: 1.6130 - acc: 0.4125
Epoch 4/20
800/800 [==============================] - 0s 41us/step - loss: 1.4445 - acc: 0.4900
Epoch 5/20
800/800 [==============================] - 0s 43us/step - loss: 1.3108 - acc: 0.5112
Epoch 6/20
800/800 [==============================] - 0s 38us/step - loss: 1.1940 - acc: 0.5950
Epoch 7/20
800/800 [==============================] - 0s 38us/step - loss: 1.1172 - acc: 0.6250
Epoch 8/20
800/800 [==============================] - 0s 38us/step - loss: 1.0246 - acc: 0.6575
Epoch 9/20
800/800 [==============================] - 0s 39us/step - loss: 0.9585 - acc: 0.6812
Epoch 10/20
800/800 [==============================] - 0s 39us/step - loss: 0.9151 - acc: 0.7150
Epoch 11/20
800/800 [===================

In [82]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 2ms/step


In [83]:
print('test_acc: ',test_acc)

test_acc:  0.675


In [84]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [88]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=50,
          batch_size=128,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)
print('Test loss:', results[0])
print('Test accuracy:', results[1])

Train on 600 samples, validate on 200 samples
Epoch 1/50
600/600 [==============================] - 2s 3ms/step - loss: 2.1512 - acc: 0.2817 - val_loss: 1.7666 - val_acc: 0.3800
Epoch 2/50
600/600 [==============================] - 0s 243us/step - loss: 1.6716 - acc: 0.4183 - val_loss: 1.5027 - val_acc: 0.4850
Epoch 3/50
600/600 [==============================] - 0s 250us/step - loss: 1.3601 - acc: 0.4933 - val_loss: 1.3406 - val_acc: 0.5500
Epoch 4/50
600/600 [==============================] - 0s 253us/step - loss: 1.1596 - acc: 0.6050 - val_loss: 1.3026 - val_acc: 0.5500
Epoch 5/50
600/600 [==============================] - 0s 230us/step - loss: 1.0174 - acc: 0.6367 - val_loss: 1.2004 - val_acc: 0.5600
Epoch 6/50
600/600 [==============================] - 0s 197us/step - loss: 0.8861 - acc: 0.6983 - val_loss: 1.1732 - val_acc: 0.6200
Epoch 7/50
600/600 [==============================] - 0s 213us/step - loss: 0.8294 - acc: 0.7000 - val_loss: 1.1601 - val_acc: 0.6000
Epoch 8/50
600/600

In [89]:
predictions = model.predict(X_test[0])

In [90]:
predictions[0].shape


(10,)

In [93]:
np.sum(predictions[0])

0.99999994

In [94]:
np.argmax(predictions[0])

8

In [16]:
x = 0 
y = 0 
def incr(x): 
    y = x + 1
    return y 
incr(5) 



6

In [14]:
x=0
y=0
def incr(x):
    y=x+1
    return y
incr(5)
print(x,y)



0 0


In [20]:
x=0
numcalls = 0 
def square(x): 
    global numcalls 
    numcalls = numcalls + 1 
    return x * x
print (square(5))
print(numcalls)


25
1


In [23]:
import time
print (time.asctime())


Mon Jun 10 16:53:47 2019


In [25]:
range(3,6)

range(3, 6)

In [28]:
zip(["a", "b", "c"], [1, 2, 3])


In [29]:
print (zip(["a", "b", "c"], [1, 2, 3]))

In [33]:
a = [1, 2]
b = [1.5, 2, a]
print (b)
print(len(a))
print(x)


[1.5, 2, [1, 2]]
2
3


In [36]:
x = [20, 21, [22]] 
x[2][0] = 3 
x[2].append(4) 
print (x) 
x[2] = 2 
print (x)


[20, 21, [3, 4]]
[20, 21, 2]
